In [4]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [5]:
# stats per game
# id: season_start_year, element
player_details = pd.read_csv('artifacts/fetched_data/get_player_details.csv')

In [6]:
# player season summary
# id: season_name, player_name
#player_hist = pd.read_csv('artifacts/fetched_data/get_player_hist.csv')

# DON'T USE

In [7]:
# player season summary from a fantasy perspective. Includes element (position)
# id: season_start_year, id
player_info = pd.read_csv('artifacts/fetched_data/get_player_info.csv')

# Use only element. Perhaps in future we could consider if they going to play or not.

# Create data

In [8]:
# fix with data
player_details = player_details.rename(columns={"element":"player_id"})
player_details['kickoff_date'] = pd.to_datetime(player_details['kickoff_time'])
player_details['kickoff_date'] = player_details['kickoff_date'].dt.strftime('%Y-%m-%d')

# add position
player_info = player_info[["season_start_year", "id", "element_type"]].rename(columns={"id":"player_id"})
player = player_details.merge(player_info, on=["season_start_year", "player_id"])
player['position'] = np.where(player.element_type == 1, "G", np.where(player.element_type == 2, "D", np.where(player.element_type == 3, "M", "F")))

# add team
game_list = pd.read_csv('artifacts/fetched_data/get_game_list.csv')
game_list = game_list[["season_start_year", "id", "team_h", "team_a"]].rename(columns={"id":"fixture"})
player = player.merge(game_list, on=["season_start_year", "fixture"])
player['team_id_season'] = np.where(player.was_home == True, player.team_h, player.team_a)
player = player.drop(["team_h", "team_a"], axis=1)

# add next_fixture
from src.components.data.transform.team_form import team_form
teams_fixture = team_form.data_setup()[["season_start_year", "team_id_season", "id", "next_id"]].rename(columns={"id":"fixture"}) # problem when player change team, then next-fixture will be the new team next fixture
player = player.merge(teams_fixture, on=["season_start_year", "team_id_season", "fixture"], how="inner")

# sorting
player = player.sort_values(["season_start_year", "player_id", "kickoff_date"]).reset_index(drop=True)
player = player.drop(["kickoff_time", "own_goals", "penalties_saved", "penalties_missed", "bonus", "bps", "ict_index", "element_type", "opponent_team"], axis=1)


In [9]:
def hourly_rate(df,features):
    for f in features:
        df["hourly_rate_"+f] = np.where(df['minutes'] > 0, (df[f] / df['minutes'])*60, 0)
    return df

In [10]:
to_hourly_rate_features = ["goals_scored","assists","clean_sheets","goals_conceded","saves"]
player = hourly_rate(player,to_hourly_rate_features)

In [11]:
def player_impact_per_game(df,features):
    team_total_minutes = df.groupby(["season_start_year", "team_id_season", "fixture"], as_index=False, dropna=False).agg(team_total_minutes = ('minutes', np.sum))
    df = df.merge(team_total_minutes, on=["season_start_year", "team_id_season", "fixture"], how="left")
    df['impact_on_game_by_minutes'] = df['minutes'] / df['team_total_minutes']

    for f in features:
        df["impact_game_"+f] = df[f] * df['impact_on_game_by_minutes']

    return df

In [12]:
to_impact_features = ["influence","creativity","threat","value","transfers_balance","selected"]
hourly_rate_features = [value for value in player.columns.to_list() if "hourly_rate" in value]
player = player_impact_per_game(player, to_impact_features + hourly_rate_features)


In [13]:
def player_rolling_last_x_games(df, features_to_mean, features_to_max, x_games=6):
    df = df.sort_values(["season_start_year", "player_id", "kickoff_date"]).reset_index(drop=True)
    
    for f in features_to_mean:
        df['rolling_mean_'+str(x_games)+'_'+f] = df.groupby(['season_start_year','player_id'])[f].rolling(window=x_games, min_periods=1).mean().reset_index(drop=True)
    
    for f in features_to_max:
        df['rolling_max_'+str(x_games)+'_'+f] = df.groupby(['season_start_year','player_id'])[f].rolling(window=x_games, min_periods=1).max().reset_index(drop=True)

    return df

In [14]:
impact_game_features = [value for value in player.columns.to_list() if "impact_game" in value]

features_to_mean = hourly_rate_features + impact_game_features
features_to_max = impact_game_features
player = player_rolling_last_x_games(player, features_to_mean, features_to_max)

# to team

In [15]:
rolling_features = [value for value in player.columns.to_list() if "rolling_" in value]
rolling_features

['rolling_mean_6_hourly_rate_goals_scored',
 'rolling_mean_6_hourly_rate_assists',
 'rolling_mean_6_hourly_rate_clean_sheets',
 'rolling_mean_6_hourly_rate_goals_conceded',
 'rolling_mean_6_hourly_rate_saves',
 'rolling_mean_6_impact_game_influence',
 'rolling_mean_6_impact_game_creativity',
 'rolling_mean_6_impact_game_threat',
 'rolling_mean_6_impact_game_value',
 'rolling_mean_6_impact_game_transfers_balance',
 'rolling_mean_6_impact_game_selected',
 'rolling_mean_6_impact_game_hourly_rate_goals_scored',
 'rolling_mean_6_impact_game_hourly_rate_assists',
 'rolling_mean_6_impact_game_hourly_rate_clean_sheets',
 'rolling_mean_6_impact_game_hourly_rate_goals_conceded',
 'rolling_mean_6_impact_game_hourly_rate_saves',
 'rolling_max_6_impact_game_influence',
 'rolling_max_6_impact_game_creativity',
 'rolling_max_6_impact_game_threat',
 'rolling_max_6_impact_game_value',
 'rolling_max_6_impact_game_transfers_balance',
 'rolling_max_6_impact_game_selected',
 'rolling_max_6_impact_game_hour

In [16]:
max_all = player.groupby(['season_start_year', 'team_id_season', 'next_id']).apply(lambda x: pd.Series({
    f'max_all_{col}': x[col].max()
    for col in rolling_features
})).reset_index()

In [17]:
desired_terms = ['saves', 'clean_sheet', 'value', 'selected', 'goals_conceded']
g_features = [element for element in rolling_features if any(term in element for term in desired_terms)]
g_features

['rolling_mean_6_hourly_rate_clean_sheets',
 'rolling_mean_6_hourly_rate_goals_conceded',
 'rolling_mean_6_hourly_rate_saves',
 'rolling_mean_6_impact_game_value',
 'rolling_mean_6_impact_game_selected',
 'rolling_mean_6_impact_game_hourly_rate_clean_sheets',
 'rolling_mean_6_impact_game_hourly_rate_goals_conceded',
 'rolling_mean_6_impact_game_hourly_rate_saves',
 'rolling_max_6_impact_game_value',
 'rolling_max_6_impact_game_selected',
 'rolling_max_6_impact_game_hourly_rate_clean_sheets',
 'rolling_max_6_impact_game_hourly_rate_goals_conceded',
 'rolling_max_6_impact_game_hourly_rate_saves']

In [18]:
max_g = player.loc[player.position == "G"].groupby(['season_start_year', 'team_id_season', 'next_id']).apply(lambda x: pd.Series({
    f'max_g_{col}': x[col].max()
    for col in g_features
})).reset_index()

In [19]:
not_desired_terms = ['saves', 'clean_sheet']
d_m_f_features = [element for element in rolling_features if not any(term in element for term in desired_terms)]
d_m_f_features

['rolling_mean_6_hourly_rate_goals_scored',
 'rolling_mean_6_hourly_rate_assists',
 'rolling_mean_6_impact_game_influence',
 'rolling_mean_6_impact_game_creativity',
 'rolling_mean_6_impact_game_threat',
 'rolling_mean_6_impact_game_transfers_balance',
 'rolling_mean_6_impact_game_hourly_rate_goals_scored',
 'rolling_mean_6_impact_game_hourly_rate_assists',
 'rolling_max_6_impact_game_influence',
 'rolling_max_6_impact_game_creativity',
 'rolling_max_6_impact_game_threat',
 'rolling_max_6_impact_game_transfers_balance',
 'rolling_max_6_impact_game_hourly_rate_goals_scored',
 'rolling_max_6_impact_game_hourly_rate_assists']

In [20]:
max_d = player.loc[player.position == "D"].groupby(['season_start_year', 'team_id_season', 'next_id']).apply(lambda x: pd.Series({
    f'max_d_{col}': x[col].max()
    for col in d_m_f_features
})).reset_index()

In [21]:
max_m = player.loc[player.position == "M"].groupby(['season_start_year', 'team_id_season', 'next_id']).apply(lambda x: pd.Series({
    f'max_m_{col}': x[col].max()
    for col in d_m_f_features
})).reset_index()

In [22]:
max_f = player.loc[player.position == "F"].groupby(['season_start_year', 'team_id_season', 'next_id']).apply(lambda x: pd.Series({
    f'max_f_{col}': x[col].max()
    for col in d_m_f_features
})).reset_index()

In [23]:
team_player_stats = max_all.merge(max_g, on=['season_start_year', 'team_id_season', 'next_id'], how="left")
team_player_stats = team_player_stats.merge(max_d, on=['season_start_year', 'team_id_season', 'next_id'], how="left")
team_player_stats = team_player_stats.merge(max_m, on=['season_start_year', 'team_id_season', 'next_id'], how="left")
team_player_stats = team_player_stats.merge(max_f, on=['season_start_year', 'team_id_season', 'next_id'], how="left")

In [24]:
team_player_stats

,season_start_year,team_id_season,next_id,max_all_rolling_mean_6_hourly_rate_goals_scored,max_all_rolling_mean_6_hourly_rate_assists,max_all_rolling_mean_6_hourly_rate_clean_sheets,max_all_rolling_mean_6_hourly_rate_goals_conceded,max_all_rolling_mean_6_hourly_rate_saves,max_all_rolling_mean_6_impact_game_influence,max_all_rolling_mean_6_impact_game_creativity,max_all_rolling_mean_6_impact_game_threat,max_all_rolling_mean_6_impact_game_value,max_all_rolling_mean_6_impact_game_transfers_balance,max_all_rolling_mean_6_impact_game_selected,max_all_rolling_mean_6_impact_game_hourly_rate_goals_scored,max_all_rolling_mean_6_impact_game_hourly_rate_assists,max_all_rolling_mean_6_impact_game_hourly_rate_clean_sheets,max_all_rolling_mean_6_impact_game_hourly_rate_goals_conceded,max_all_rolling_mean_6_impact_game_hourly_rate_saves,max_all_rolling_max_6_impact_game_influence,max_all_rolling_max_6_impact_game_creativity,max_all_rolling_max_6_impact_game_threat,max_all_rolling_max_6_impact_game_value,max_all_rolling_max_6_impact_game_transfers_balance,max_all_rolling_max_6_impact_game_selected,max_all_rolling_max_6_impact_game_hourly_rate_goals_scored,max_all_rolling_max_6_impact_game_hourly_rate_assists,max_all_rolling_max_6_impact_game_hourly_rate_clean_sheets,max_all_rolling_max_6_impact_game_hourly_rate_goals_conceded,max_all_rolling_max_6_impact_game_hourly_rate_saves,max_g_rolling_mean_6_hourly_rate_clean_sheets,max_g_rolling_mean_6_hourly_rate_goals_conceded,max_g_rolling_mean_6_hourly_rate_saves,max_g_rolling_mean_6_impact_game_value,max_g_rolling_mean_6_impact_game_selected,max_g_rolling_mean_6_impact_game_hourly_rate_clean_sheets,max_g_rolling_mean_6_impact_game_hourly_rate_goals_conceded,max_g_rolling_mean_6_impact_game_hourly_rate_saves,max_g_rolling_max_6_impact_game_value,max_g_rolling_max_6_impact_game_selected,max_g_rolling_max_6_impact_game_hourly_rate_clean_sheets,max_g_rolling_max_6_impact_game_hourly_rate_goals_conceded,max_g_rolling_max_6_impact_game_hourly_rate_saves,max_d_rolling_mean_6_hourly_rate_goals_scored,max_d_rolling_mean_6_hourly_rate_assists,max_d_rolling_mean_6_impact_game_influence,max_d_rolling_mean_6_impact_game_creativity,max_d_rolling_mean_6_impact_game_threat,max_d_rolling_mean_6_impact_game_transfers_balance,max_d_rolling_mean_6_impact_game_hourly_rate_goals_scored,max_d_rolling_mean_6_impact_game_hourly_rate_assists,max_d_rolling_max_6_impact_game_influence,max_d_rolling_max_6_impact_game_creativity,max_d_rolling_max_6_impact_game_threat,max_d_rolling_max_6_impact_game_transfers_balance,max_d_rolling_max_6_impact_game_hourly_rate_goals_scored,max_d_rolling_max_6_impact_game_hourly_rate_assists,max_m_rolling_mean_6_hourly_rate_goals_scored,max_m_rolling_mean_6_hourly_rate_assists,max_m_rolling_mean_6_impact_game_influence,max_m_rolling_mean_6_impact_game_creativity,max_m_rolling_mean_6_impact_game_threat,max_m_rolling_mean_6_impact_game_transfers_balance,max_m_rolling_mean_6_impact_game_hourly_rate_goals_scored,max_m_rolling_mean_6_impact_game_hourly_rate_assists,max_m_rolling_max_6_impact_game_influence,max_m_rolling_max_6_impact_game_creativity,max_m_rolling_max_6_impact_game_threat,max_m_rolling_max_6_impact_game_transfers_balance,max_m_rolling_max_6_impact_game_hourly_rate_goals_scored,max_m_rolling_max_6_impact_game_hourly_rate_assists,max_f_rolling_mean_6_hourly_rate_goals_scored,max_f_rolling_mean_6_hourly_rate_assists,max_f_rolling_mean_6_impact_game_influence,max_f_rolling_mean_6_impact_game_creativity,max_f_rolling_mean_6_impact_game_threat,max_f_rolling_mean_6_impact_game_transfers_balance,max_f_rolling_mean_6_impact_game_hourly_rate_goals_scored,max_f_rolling_mean_6_impact_game_hourly_rate_assists,max_f_rolling_max_6_impact_game_influence,max_f_rolling_max_6_impact_game_creativity,max_f_rolling_max_6_impact_game_threat,max_f_rolling_max_6_impact_game_transfers_balance,max_f_rolling_max_6_impact_game_hourly_rate_goals_scored,max_f_rolling_max_6_impact_game_hourly_rate_assists
0,19,1,11.0,

In [27]:
team_player_stats.groupby(["season_start_year", "team_id_season", "next_id"], dropna=False, as_index=False).size().sort_values("size")

,season_start_year,team_id_season,next_id,size
0,19,1,11.0,1
1488,21,1,93.0,1
1487,21,1,81.0,1
1486,21,1,71.0,1
1485,21,1,61.0,1
...,...,...,...,...
741,20,1,23.0,1
742,20,1,29.0,1
743,20,1,44.0,1
737,19,20,352.0,1
